In [12]:
# Import library:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

# Définition du chemin d'accès:
### Céline
# %cd C:\Users\c.doussot\Desktop\Data Analyst\GitHub\ParisPyVelib_Datas
### Hermine

### Tarik
%cd C:\Users\anouart\Documents\JupyterNotebook\PyVelib\Data

C:\Users\anouart\Documents\JupyterNotebook\PyVelib\Data


- Une idée serait de faire la modélisation sur les comptages horaires moyen par jour et non par heure. Cela simplifiera le nombre de données et permettra d'intégrer facilement les données météos.

## Préparation des données

In [13]:
df = pd.read_csv('2018-2021_donnees-velib-meteo_hour.csv')

In [14]:
# selection d'un seul compteur
df = df[df.Address == "89 BD DE MAGENTA"]

# supression des colonnes non utile pour le modele
df = df.drop(['Unnamed: 0', 'Coord', 'Coord_full', 'City_meteo', 'Latitude', 'Longitude', 'vacances_zone_c', 'Address', 
              'Direction', 'Source', 'Id', 'Date_instal'], axis = 1)

# passage de la colonne date en datetime et passage en ms
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%dT%H:%M:%S')
df['Date'] = df.Date.astype('int64') // 10**9      # réciproque : df['Date'] = pd.to_datetime(df['Date'], unit='s')

# reset de l'index
df = df.reset_index(drop=True)

# list des variable catégorielle
cat_df = df.select_dtypes(include='object')
cat_df = cat_df.columns.tolist()

# Transformation des variable catégorielle en variable indicatrices
df = pd.get_dummies(df, columns=cat_df)

# affichage des 5 première lignes du df
df.head()

,Count_by_hour,Date,Y_Date_Count,M_Date_Count,D_Date_Count,Dweek_Date_Count,H_Date_Count,Y_Date_Instal,M_Date_Instal,D_Date_Instal,...,nom_jour_ferie_Assomption,nom_jour_ferie_Jour de Noël,nom_jour_ferie_Lundi de Pentecôte,nom_jour_ferie_Lundi de Pâques,nom_jour_ferie_None,nom_jour_ferie_Toussaint,Confinement_id_Confinement 1,Confinement_id_Confinement 2,Confinement_id_Confinement 3,Confinement_id_None
0,0.0,1543377600,2018.0,11.0,28.0,2.0,4.0,2018.0,11.0,28.0,...,0,0,0,0,1,0,0,0,0,1
1,0.0,1543388400,2018.0,11.0,28.0,2.0,7.0,2018.0,11.0,28.0,...,0,0,0,0,1,0,0,0,0,1
2,247.0,1543399200,2018.0,11.0,28.0,2.0,10.0,2018.0,11.0,28.0,...,0,0,0,0,1,0,0,0,0,1
3,96.0,1543410000,2018.0,11.0,28.0,2.0,13.0,2018.0,11.0,28.0,...,0,0,0,0,1,0,0,0,0,1
4,95.0,1543420800,2018.0,11.0,28.0,2.0,16.0,2018.0,11.0,28.0,...,0,0,0,0,1,0,0,0,0,1


In [15]:
# selection de la target et des feats
target = df['Count_by_hour']
feats = df.drop(['Count_by_hour'], axis=1)

In [16]:
# vérification du type des variables pour feats
feats['High_ice'] = feats['High_ice'].fillna(0)
feats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7268 entries, 0 to 7267
Data columns (total 37 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Date                                   7268 non-null   int64  
 1   Y_Date_Count                           7268 non-null   float64
 2   M_Date_Count                           7268 non-null   float64
 3   D_Date_Count                           7268 non-null   float64
 4   Dweek_Date_Count                       7268 non-null   float64
 5   H_Date_Count                           7268 non-null   float64
 6   Y_Date_Instal                          7268 non-null   float64
 7   M_Date_Instal                          7268 non-null   float64
 8   D_Date_Instal                          7268 non-null   float64
 9   T°C                                    7268 non-null   float64
 10  Precip_last3h                          7268 non-null   float64
 11  HR% 

## SVM

In [24]:
target = pd.qcut(target, 4, labels = [0, 1, 2, 3])

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.05, random_state=101)

In [27]:
# preprocessing

from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [28]:
from sklearn import svm

# Création d'un classifieur clf sans paramètres
clf = svm.SVC()
# Entrainement de l'algorithme sur l'ensemble d'entrainement
clf.fit(X_train_scaled, y_train)

SVC()

In [29]:
from sklearn import model_selection

# Recherche des meilleurs hyperparamètres

parametres = {'C':[0.1,1,10,50], 'kernel':['rbf','linear']}

grid_clf = model_selection.GridSearchCV(estimator=clf, param_grid=parametres)

grille = grid_clf.fit(X_train_scaled,y_train)

print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']])

                           params  mean_test_score
0     {'C': 0.1, 'kernel': 'rbf'}         0.576186
1  {'C': 0.1, 'kernel': 'linear'}         0.587341
2       {'C': 1, 'kernel': 'rbf'}         0.629345
3    {'C': 1, 'kernel': 'linear'}         0.590238
4      {'C': 10, 'kernel': 'rbf'}         0.693366
5   {'C': 10, 'kernel': 'linear'}         0.590528
6      {'C': 50, 'kernel': 'rbf'}         0.717844
7   {'C': 50, 'kernel': 'linear'}         0.590383


In [21]:
print('Le modèle qui fournit la meilleur prédiction à pour paramètres :',grid_clf.best_params_)

Le modèle qui fournit la meilleur prédiction à pour paramètres : {'C': 10, 'kernel': 'rbf'}


In [30]:
# Prédiction des features test et création de la matrice de confusion

clf = svm.SVC(C=10, kernel='rbf')
clf.fit(X_train_scaled, y_train)

y_pred = grid_clf.predict(X_test_scaled)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,0,1,2,3
Classe réelle,,,,
0,64,12,1,0
1,14,61,9,1
2,1,13,55,20
3,0,3,16,94
